# Deploying model into AWS cloud environment using SageMaker.

Steps:
- [ ] Upload model to S3 bucket
- [ ] Convert Keras to Protobuf format from H5
- [ ] Deploy an endpoint in SageMaker
- [ ] Establish a lambda function triggered by API gateway (Code provided)
- [ ] Notebook entry to generate S3 signed urls
- [ ] Upload file to S3 using signed urls
- [ ] Infer endpoint to infer on the uploaded file
- [ ] Visualize the image and results
- [ ] Add descriptions to the notebook

In [ ]:
# Upgrade available version of Sagemaker

!pip install -U sagemaker tensorflow==2.9.1 numpy

In [ ]:
# import SageMaker and TensorFlow model to load the model from S3 bucket

import SageMaker
from sagemaker.tensorflow import TensorFlowModel


In [ ]:
# TensorFlow imports needed for model conversion.
import tarfile
import TensorFlow as tf
import tensorflow.compat.v1.keras.backend as K

from tensorflow.python.saved_model import builder as model_builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants


from sagemaker import get_execution_role
from sagemaker import Session
from sagemaker.tensorflow.serving import Model


In [ ]:
# Account related constants

BUCKET = "smoke-dataset-bucket"
EXECUTION_ROLE = 'arn:aws:iam::574347909231:role/igarss-sagemaker-role'


In [ ]:
model_filename = "<username>_smoke_wmts_ref_3layer.h5" # <model filename> # smoke_wmts_ref_3layer.h5
framework_version = "2.9.1" # <tensorflow version> # 2.9.1


In [ ]:
role = get_execution_role()
print(role)
session = Session()
session.download_data('.', BUCKET, key_prefix=f"<username>/{model_filename}", extra_args=None)

## Convert Keras HDF5 format to Protobuf format

We used Keras to train and save our model. This is not recognized by the SageMaker processes as a native model format. Hence, we will need to change the format to Protocol Buffer format which is easier to be read by SageMaker.

The following methods help convert the current format to Protobuf format and prepare a package which can then be deployed to the SageMaker Endpoint

In [ ]:
from tensorflow.keras.models import load_model

MODEL_VERSION = '1'
EXPORT_DIRECTORY = 'export/Servo/{}'
MODEL_ARCHIVE = '<name>model.tar.gz'


def upload_model(model_archive=MODEL_ARCHIVE):
    """
    Method to upload Protobuf based model to S3.
    Args:
        model_archive(str): name of the model archive.
    Returns:
        model_data: details of the model that was uploaded
    """
    with tarfile.open(model_archive, mode='w:gz') as archive:
        archive.add('export', recursive=True) 

    role = get_execution_role()
    sess = Session()
    bucket = sess.default_bucket()
    
    # upload model artifacts to S3
    return sess.upload_data(
            path=model_archive, 
            key_prefix='model'
        )


def convert_to_proto_buf(model_name):
    """
    Converts Keras based model to Protocol Buffer format recognized by TensorFlow
    Args:
        model_name: File path to the model we want to convert.
    Return: 
        signature: Input/Output signature of the loaded model
    """
    loaded_model = load_model(model_name)
    export_dir =  EXPORT_DIRECTORY.format(MODEL_VERSION)
    builder = model_builder.SavedModelBuilder(export_dir)
    tf.compat.v1.disable_eager_execution()
    signature = predict_signature_def(
            inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output}
        )
    builder.add_meta_graph_and_variables(
    sess=K.get_session(), tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()
    return signature



In [ ]:
INSTANCE_TYPE = 'ml.t2.medium'

convert_to_proto_buf(model_filename)
model_data = upload_model()


### Define SageMaker model based on uploaded model

Once the model is uploaded, you need to let SageMaker know where to get the model and provide an execution role which will have access to the S3 bucket where the model was pushed.

In [ ]:
sagemaker_model = TensorFlowModel(
        model_data=model_data, 
        framework_version='2.8',
        role=EXECUTION_ROLE
    )


### Deploy the model

Now, you deploy the model into an endpoint. Endpoints are how models are hosted. Once the endpoints are established, you can add a lambda function which interacts with the endpoint and use it as the API backend.

Here, you are using `ml.t2.medium` instance to host your SageMaker endpoint. If there is a higher memory requirement, you can change the type of instance depending on your needs.

_Note: Change `<name>` to reflect your name._

In [ ]:
%%time
uncompiled_predictor = sagemaker_model.deploy(
        initial_instance_count=1, 
        instance_type=INSTANCE_TYPE,
        endpoint_name="<name>-prediction-endpoint"
    )   

### Compare the local and deployed version of the model.

The endpoint should be available for you to use in a couple of minutes. Once the model is deployed, you can compare the inferences between Keras version and Protobuf deployed version.

Here, you are just creating random values and infering on the data point using both versions of the model. The difference between the inferences should be negligible.

In [ ]:
# Locally load the model for comparison
loaded_model = load_model(model_filename)

In [ ]:
import numpy as np

data = np.random.randn(1, 256, 256, 6)

In [ ]:
deployed_model_preds = uncompiled_predictor.predict(data)
original_model_preds = loaded_model.predict(data)

In [ ]:
difference = deployed_model_preds['predictions'] - original_model_preds